In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash
from neqsim.process import (
    stream, separator, separator3phase, heater, 
    valve, compressor, pump, mixer, splitter,
    clearProcess, run, getProcess
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1. Create Well Stream Fluid

A typical well stream contains hydrocarbons and water.

In [ ]:
# Create oil/gas/water fluid
wellfluid = fluid("srk")
wellfluid.addComponent("nitrogen", 0.5)
wellfluid.addComponent("CO2", 1.5)
wellfluid.addComponent("methane", 65.0)
wellfluid.addComponent("ethane", 8.0)
wellfluid.addComponent("propane", 5.0)
wellfluid.addComponent("i-butane", 1.0)
wellfluid.addComponent("n-butane", 2.5)
wellfluid.addComponent("i-pentane", 1.0)
wellfluid.addComponent("n-pentane", 1.5)
wellfluid.addComponent("n-hexane", 2.0)
wellfluid.addComponent("n-heptane", 3.0)
wellfluid.addComponent("n-octane", 2.0)
wellfluid.addComponent("water", 7.0)  # Produced water
wellfluid.setMixingRule(2)
wellfluid.setMultiPhaseCheck(True)

# Wellhead conditions
P_wellhead = 80.0  # bara
T_wellhead = 75.0  # °C

wellfluid.setTemperature(T_wellhead, "C")
wellfluid.setPressure(P_wellhead, "bara")
wellfluid.setTotalFlowRate(50000, "kg/hr")

TPflash(wellfluid)

print(f"Wellhead conditions: {P_wellhead} bara, {T_wellhead}°C")
print(f"Total flow rate: 50,000 kg/hr")
print(f"Number of phases: {wellfluid.getNumberOfPhases()}")

## 2. Two-Phase Separator

A basic gas/liquid separator.

In [ ]:
clearProcess()

# Create inlet stream
wellfluid.setTemperature(T_wellhead, "C")
wellfluid.setPressure(P_wellhead, "bara")
wellfluid.setTotalFlowRate(50000, "kg/hr")

inlet = stream(wellfluid, "well stream")

# Two-phase separator at 60 bara
sep1 = separator(inlet, "HP separator")

# Add valve to reduce pressure
inlet_valve = valve(inlet, 60.0, "inlet choke")

run()

In [ ]:
# Display separator results
print("Two-Phase Separator Results:")
print("=" * 50)

print(f"\nInlet Stream:")
print(f"  Pressure: {inlet.getPressure():.1f} bara")
print(f"  Temperature: {inlet.getTemperature() - 273.15:.1f}°C")
print(f"  Mass flow: {inlet.getFlowRate('kg/hr'):.0f} kg/hr")

print(f"\nGas Outlet:")
gas_out = sep1.getGasOutStream()
print(f"  Mass flow: {gas_out.getFlowRate('kg/hr'):.0f} kg/hr")
print(f"  Molar flow: {gas_out.getFlowRate('kmol/hr'):.1f} kmol/hr")

print(f"\nLiquid Outlet:")
liq_out = sep1.getLiquidOutStream()
print(f"  Mass flow: {liq_out.getFlowRate('kg/hr'):.0f} kg/hr")
print(f"  Molar flow: {liq_out.getFlowRate('kmol/hr'):.1f} kmol/hr")

## 3. Three-Phase Separator

Separates gas, oil, and water into three streams.

In [ ]:
clearProcess()

# Create CPA fluid for water handling
wellfluid_cpa = fluid("cpa")
wellfluid_cpa.addComponent("nitrogen", 0.5)
wellfluid_cpa.addComponent("CO2", 1.5)
wellfluid_cpa.addComponent("methane", 60.0)
wellfluid_cpa.addComponent("ethane", 8.0)
wellfluid_cpa.addComponent("propane", 5.0)
wellfluid_cpa.addComponent("i-butane", 1.0)
wellfluid_cpa.addComponent("n-butane", 2.5)
wellfluid_cpa.addComponent("n-pentane", 2.0)
wellfluid_cpa.addComponent("n-hexane", 2.5)
wellfluid_cpa.addComponent("n-heptane", 4.0)
wellfluid_cpa.addComponent("water", 13.0)  # Higher water cut
wellfluid_cpa.setMixingRule(10)  # CPA mixing rule
wellfluid_cpa.setMultiPhaseCheck(True)

wellfluid_cpa.setTemperature(T_wellhead, "C")
wellfluid_cpa.setPressure(P_wellhead, "bara")
wellfluid_cpa.setTotalFlowRate(100000, "kg/hr")

# Create process
inlet_3ph = stream(wellfluid_cpa, "well stream")

# Three-phase separator
sep_3phase = separator3phase(inlet_3ph, "3-phase separator")

run()

In [ ]:
# Display 3-phase separator results
print("Three-Phase Separator Results:")
print("=" * 50)

print(f"\nInlet:")
print(f"  Total flow: {inlet_3ph.getFlowRate('kg/hr'):.0f} kg/hr")

print(f"\nGas Outlet:")
gas_3ph = sep_3phase.getGasOutStream()
print(f"  Mass flow: {gas_3ph.getFlowRate('kg/hr'):.0f} kg/hr")

print(f"\nOil Outlet:")
oil_3ph = sep_3phase.getOilOutStream()
print(f"  Mass flow: {oil_3ph.getFlowRate('kg/hr'):.0f} kg/hr")

print(f"\nWater Outlet:")
water_3ph = sep_3phase.getWaterOutStream()
print(f"  Mass flow: {water_3ph.getFlowRate('kg/hr'):.0f} kg/hr")

## 4. Multi-Stage Separation Train

Typical offshore: HP → MP → LP → Stock Tank

In [ ]:
clearProcess()

# Fresh fluid
wellfluid.setTemperature(T_wellhead, "C")
wellfluid.setPressure(P_wellhead, "bara")
wellfluid.setTotalFlowRate(50000, "kg/hr")

# Build separation train
well_stream = stream(wellfluid, "well stream")

# HP Separator (60 bara)
choke1 = valve(well_stream, 60.0, "HP choke")
hp_sep = separator(choke1.getOutletStream(), "HP separator")

# MP Separator (15 bara)
choke2 = valve(hp_sep.getLiquidOutStream(), 15.0, "MP choke")
mp_sep = separator(choke2.getOutletStream(), "MP separator")

# LP Separator (3 bara)
choke3 = valve(mp_sep.getLiquidOutStream(), 3.0, "LP choke")
lp_sep = separator(choke3.getOutletStream(), "LP separator")

run()

In [ ]:
# Display multi-stage results
print("Multi-Stage Separation Results:")
print("=" * 60)

stages = [
    ("HP Separator", hp_sep, 60),
    ("MP Separator", mp_sep, 15),
    ("LP Separator", lp_sep, 3),
]

print(f"\n{'Stage':<15} {'P [bara]':<10} {'Gas [kg/hr]':<15} {'Liquid [kg/hr]':<15}")
print("-" * 60)

total_gas = 0
for name, sep, pressure in stages:
    gas_flow = sep.getGasOutStream().getFlowRate('kg/hr')
    liq_flow = sep.getLiquidOutStream().getFlowRate('kg/hr')
    total_gas += gas_flow
    print(f"{name:<15} {pressure:<10} {gas_flow:<15.0f} {liq_flow:<15.0f}")

final_oil = lp_sep.getLiquidOutStream().getFlowRate('kg/hr')
print(f"\nTotal gas produced: {total_gas:.0f} kg/hr")
print(f"Final oil product: {final_oil:.0f} kg/hr")
print(f"GOR: {total_gas / final_oil:.2f} kg gas / kg oil")

## 5. Optimization: Finding Optimal Stage Pressures

The goal is to maximize oil recovery (minimize flash losses).

In [ ]:
def run_separation_train(p_hp, p_mp, p_lp):
    """Run 3-stage separation and return oil production."""
    clearProcess()
    
    wellfluid.setTemperature(T_wellhead, "C")
    wellfluid.setPressure(P_wellhead, "bara")
    wellfluid.setTotalFlowRate(50000, "kg/hr")
    
    inlet = stream(wellfluid, "inlet")
    
    v1 = valve(inlet, p_hp, "v1")
    s1 = separator(v1.getOutletStream(), "s1")
    
    v2 = valve(s1.getLiquidOutStream(), p_mp, "v2")
    s2 = separator(v2.getOutletStream(), "s2")
    
    v3 = valve(s2.getLiquidOutStream(), p_lp, "v3")
    s3 = separator(v3.getOutletStream(), "s3")
    
    run()
    
    return s3.getLiquidOutStream().getFlowRate('kg/hr')

# Scan MP pressure with fixed HP=60, LP=3
mp_pressures = np.arange(8, 35, 2)
oil_flows = []

for mp_p in mp_pressures:
    oil = run_separation_train(60, mp_p, 3)
    oil_flows.append(oil)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(mp_pressures, oil_flows, 'b-o', linewidth=2, markersize=6)
plt.xlabel('MP Separator Pressure [bara]', fontsize=12)
plt.ylabel('Oil Production [kg/hr]', fontsize=12)
plt.title('Oil Recovery vs MP Separator Pressure\n(HP=60 bara, LP=3 bara)', fontsize=14)
plt.grid(True, alpha=0.3)

# Find optimum
opt_idx = np.argmax(oil_flows)
opt_mp = mp_pressures[opt_idx]
opt_oil = oil_flows[opt_idx]
plt.axvline(x=opt_mp, color='r', linestyle='--', label=f'Optimum: {opt_mp} bara')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Optimal MP pressure: {opt_mp} bara")
print(f"Maximum oil production: {opt_oil:.0f} kg/hr")

## 6. Separator Design Considerations

In [ ]:
print("""
SEPARATOR DESIGN GUIDELINES
===========================

1. SEPARATOR TYPES
   ├── Horizontal: Large liquid volumes, 3-phase, slugging
   ├── Vertical: Small footprint, gas dominated, scrubbers
   └── Spherical: High pressure, compact (less common)

2. SIZING CRITERIA
   Gas Capacity:
   - Liquid droplet settling velocity
   - Typically size for 100-500 micron droplets
   - Mist extractor for smaller droplets
   
   Liquid Capacity:
   - Retention time: 2-5 minutes typical
   - Slug handling: May need surge volume
   - Oil/water separation: 5-10 minutes for 3-phase

3. OPERATING PRESSURE SELECTION
   - HP: Minimize flashing, reduce compressor work
   - LP: Maximize light ends recovery
   - Optimal stages: Equal pressure ratios

4. TEMPERATURE EFFECTS
   - Higher T: More flashing, lighter oil
   - Lower T: Less flashing, heavier oil
   - May need heating for viscous oils
   
5. INTERNALS
   - Inlet device: Momentum breaker
   - Mist extractor: Vane pack or mesh pad
   - Weir plate: For 3-phase separation
   - Level control: For interface management
""")

## 7. Process Flow Diagram

In [ ]:
print("""
TYPICAL OFFSHORE SEPARATION TRAIN
==================================

                                    To Flare/Compression
                                          ↑
                                    ┌─────┴─────┐
     ┌──────────┐     ┌──────────┐  │    HP     │
     │  Well    │     │  Inlet   │  │    Gas    │
──▶  │  Choke   │ ──▶ │  Heater  │ ──────────────
     └──────────┘     └──────────┘       │
                                   ┌─────┴─────┐
                                   │    HP     │  60 bara
                                   │ Separator │
                                   └─────┬─────┘
                                         │ Liquid
                                   ┌─────▼─────┐
                               ────│   Valve   │────
                                   └───────────┘
                                         │
                         ┌───────────────┼───────────────┐
                         │               │               │
                    MP Gas ↑       ┌─────▼─────┐    LP Gas ↑
                    To Comp.       │    MP     │    To Comp.
                                   │ Separator │  15 bara
                                   └─────┬─────┘
                                         │
                                   ┌─────▼─────┐
                                   │    LP     │  3 bara
                                   │ Separator │
                                   └─────┬─────┘
                                         │
                                         ▼
                                   To Storage/Export
""")

## Summary

This notebook covered:

- **Two-phase separation:** Gas/liquid splitting
- **Three-phase separation:** Gas/oil/water splitting
- **Multi-stage trains:** HP → MP → LP staging
- **Pressure optimization:** Finding optimal stage pressures
- **Design guidelines:** Sizing and selection criteria

**Key insights:**
- Optimal stage pressures maximize oil recovery
- Equal pressure ratios between stages is a good starting point
- Three-phase separators are essential when water is present
- Retention time is critical for liquid separation